In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!pip3 install pytorch_pretrained_bert

In [1]:
!pip install allennlp==v0.9.0

     |████████████████████████████████| 7.6MB 1.3MB/s 
     |████████████████████████████████| 5.3MB 57.2MB/s 
     |████████████████████████████████| 133kB 57.9MB/s 
     |████████████████████████████████| 266kB 53.3MB/s 
     |████████████████████████████████| 245kB 51.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 30.9MB 100kB/s 
     |████████████████████████████████| 204kB 51.5MB/s 
     |████████████████████████████████| 163kB 22.7MB/s 
     |████████████████████████████████| 235kB 52.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 2.1MB 51.6MB/s 
     |████████████████████████████████| 3.2MB 44.7MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.16.0-cp36-

In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 675kB 22.0MB/s 
     |████████████████████████████████| 3.8MB 44.0MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=b38f59e60b5976abe3e3f634f89313fb6266554a9004abba45e32e73c7886192
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=aaafb7c93805d2dd09ba0f2b66ba0d49d7354e6d17153dd2677405a0e19e548b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class WiCNet(nn.Module):
	def __init__(self, batch_size=32, embedding_length=1024, num_labels=2):
		super(WiCNet, self).__init__()

		self.wicnet = nn.Sequential(nn.Linear(2*embedding_length, 1000),
			nn.Softmax(),
      nn.Dropout(),
			nn.Linear(1000, num_labels),
			nn.Softmax()
			)

	def forward(self, word1, word2):
		word_concat = torch.cat([word1, word2], 1)
		return self.wicnet(word_concat)

In [8]:
from pytorch_pretrained_bert import BertTokenizer, BertModel
bert_model = BertModel.from_pretrained('bert-large-uncased')
bert_model = bert_model.cuda()
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
bert_base = BertModel.from_pretrained('bert-base-uncased')
bert_base = bert_model.cuda()
tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')
    

100%|██████████| 231508/231508 [00:00<00:00, 615448.29B/s]


In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')

In [11]:
import numpy as np 
import torch
from allennlp.commands.elmo import ElmoEmbedder
elmo = ElmoEmbedder(cuda_device=0) 

100%|██████████| 374434792/374434792 [00:24<00:00, 15476409.11B/s]


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class SelfAttention(nn.Module):
	def __init__(self, batch_size=32, hidden_size=200, embedding_length=768):
		super(SelfAttention, self).__init__()

		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""

		self.batch_size = batch_size
		self.hidden_size = hidden_size

		self.bilstm = nn.LSTM(embedding_length, hidden_size, 2, dropout=0.2, bidirectional=True)
		# We will use da = 350, r = 30 & penalization_coeff = 1 as per given in the self-attention original ICLR paper
		self.W_s1 = nn.Linear(2*hidden_size, 350)
		self.W_s2 = nn.Linear(350, 30)
		self.fc_layer = nn.Linear(30*2*hidden_size, 2000)

	def attention_net(self, lstm_output):

		"""
		Now we will use self attention mechanism to produce a matrix embedding of the input sentence in which every row represents an
		encoding of the inout sentence but giving an attention to a specific part of the sentence. We will use 30 such embedding of 
		the input sentence and then finally we will concatenate all the 30 sentence embedding vectors and connect it to a fully 
		connected layer of size 2000 which will be connected to the output layer of size 2 returning logits for our two classes i.e., 
		pos & neg.
		Arguments
		---------
		lstm_output = A tensor containing hidden states corresponding to each time step of the LSTM network.
		---------
		Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
				  attention to different parts of the input sentence.
		Tensor size : lstm_output.size() = (batch_size, num_seq, 2*hidden_size)
					  attn_weight_matrix.size() = (batch_size, 30, num_seq)
		"""
		attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
		attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
		attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)

		return attn_weight_matrix

	def forward(self, input, batch_size=None):

		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class.
		
		"""

		#input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(4, self.batch_size, self.hidden_size).cuda())
		else:
			h_0 = Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())
			c_0 = Variable(torch.zeros(4, batch_size, self.hidden_size).cuda())

		output, (h_n, c_n) = self.bilstm(input, (h_0, c_0))
		output = output.permute(1, 0, 2)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		# h_n.size() = (1, batch_size, hidden_size)
		# c_n.size() = (1, batch_size, hidden_size)
		attn_weight_matrix = self.attention_net(output)
		# attn_weight_matrix.size() = (batch_size, r, num_seq)
		# output.size() = (batch_size, num_seq, 2*hidden_size)
		hidden_matrix = torch.bmm(attn_weight_matrix, output)
		# hidden_matrix.size() = (batch_size, r, 2*hidden_size)
		# Let's now concatenate the hidden_matrix and connect it to the fully connected layer.
		fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
		# logits.size() = (batch_size, output_size)

		return fc_out

class LSTM1(nn.Module):
	def __init__(self, batch_size=256, hidden_size=200, embedding_length=768, num_labels=3):
		super(LSTM1, self).__init__()

		self.premise_net = SelfAttention(batch_size, hidden_size, embedding_length)
		self.hypothesis_net = SelfAttention(batch_size, hidden_size, embedding_length)
		self.fc = nn.Linear(4000, 3)

	def forward(self, premise, hypothesis):
		rep1 = self.premise_net(premise)
		rep2 = self.hypothesis_net(hypothesis)
		rep = torch.cat([rep1, rep2], 1)
		return self.fc(rep)

In [0]:
from sentence_transformers import SentenceTransformer
def pad_tensor(vec, pad, dim):
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)	

def predict_bert(premise, hypothesis):
    lstm = LSTM1(batch_size = 1).to(device)
    lstm.load_state_dict(torch.load("/content/drive/My Drive/IG wala/checkpoints/networkbertlstm1_train_epoch_27.ckpt"))

    marked_text = "[CLS] " + premise + " [SEP]"
    tokenized_text = tokenizer_base.tokenize(marked_text)
    indexed_tokens = tokenizer_base.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    bert_base.eval()
    with torch.no_grad():
      encoded_layers, _ = bert_bese(tokens_tensor.to(device), segments_tensors.to(device))
      embedding = encoded_layers[11][0]
      premise = np.array(embedding.cpu())
    marked_text = "[CLS] " + hypothesis + " [SEP]"
    tokenized_text = tokenizer_base.tokenize(marked_text)
    indexed_tokens = tokenizer_base.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    bert_base.eval()
    with torch.no_grad():
      encoded_layers, _ = bert_base(tokens_tensor.to(device), segments_tensors.to(device))
      embedding = encoded_layers[11][0]
      hypothesis = np.array(embedding.cpu())
    premise = torch.tensor(premise).type(torch.FloatTensor)
    hypothesis = torch.tensor(hypothesis).type(torch.FloatTensor)
    premise = pad_tensor(premise, 250, 0)
    hypothesis = pad_tensor(hypothesis, 250, 0)
    premise = premise.unsqueeze(0)
    hypothesis = hypothesis.unsqueeze(0)
    with torch.no_grad():
      y_pred = lstm(premise.to(device), hypothesis.to(device))
    return y_pred

#def predict_elmo(premise, hypothesis):
#  lstm = LSTM1(batch_size = 1).to(device)
#  lstm.load_state_dict(torch.load("/content/drive/My Drive/IG wala/checkpoints/networkbertlstm1_train_epoch_27.ckpt"))

#  embedding = elmo.embed_sentence(premise.split())
#  premise_embedding = np.average(embedding, axis = 0)
  
#  embedding = elmo.embed_sentence(hypothesis.split())
#  hypothesis_embedding = np.average(embedding, axis = 0)

#  premise = torch.tensor(premise_embedding).type(torch.FloatTensor)
#  hypothesis = torch.tensor(hypothesis_embedding).type(torch.FloatTensor)
#  premise = pad_tensor(premise, 250, 0)
#  hypothesis = pad_tensor(hypothesis, 250, 0)
#  premise = premise.unsqueeze(0)
#  hypothesis = hypothesis.unsqueeze(0)
#  y_pred = lstm(premise.to(device), hypothesis.to(device))
#  return y_pred

def predict_roberta(premise, hypothesis):
  roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
  roberta.eval()
  with torch.no_grad():
    tokens = roberta.encode(premise, hypothesis)
    prediction = roberta.predict('mnli', tokens).argmax().item()
  return prediction

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [39]:
import nltk.data
nltk.download('punkt')
tokenizer1 = nltk.data.load('tokenizers/punkt/english.pickle')
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    #text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    text = " ".join(text)
    #text = text.split()
    '''newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)'''
    #text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    #text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    #text = re.sub(r'\.',' . ',text)
    text = text.replace("-", '')
    return text

def sort_by_values_len(dict):
  dict_len= {key: len(value) for key, value in dict.items()}
  import operator
  sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
  sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
  return sorted_dict

def dominating_set(st, sentences, graph):
  dom_set = []
  for token in st:
    dom_set.extend(graph[token])
  if(dom_set.sort() == sentences.sort()):
    return True
  else:
    return False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
print("Enter first sentence")
sentence1 = "The bank won't accept cash on Saturday." #input()
print("Enter second sentence")
sentence2 = "The river overflowed it's bank." #input()
print("Enter index of required word in sentence 1")
sentence1_index = 1 #int(input())
print("Enter index of required word in sentence 2")
sentence2_index = 4 #int(input())

Enter first sentence
Enter second sentence
Enter index of required word in sentence 1
Enter index of required word in sentence 2


In [10]:
marked_text = "[CLS] " + sentence1 + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
bert_model.eval()
with torch.no_grad():
	encoded_layers, _ = bert_model(tokens_tensor.to(device), segments_tensors.to(device))
	embedding = encoded_layers[11][0]
	embedding1 = embedding[sentence1_index]

marked_text = "[CLS] " + sentence2 + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
bert_model.eval()
with torch.no_grad():
	encoded_layers, _ = bert_model(tokens_tensor.to(device), segments_tensors.to(device))
	embedding = encoded_layers[11][0]
	embedding2 = embedding[sentence2_index]

embedding1, embedding2 = torch.tensor(embedding1).type(torch.FloatTensor), torch.tensor(embedding2).type(torch.FloatTensor)
embedding1 = embedding1.unsqueeze(0)
embedding2 = embedding2.unsqueeze(0)
dnn = WiCNet(batch_size = 1).to(device)
dnn.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/checkpoints/networkwicbert_train_epoch_59.ckpt"))

y_pred = dnn(embedding1.to(device), embedding2.to(device))
_, predicted = y_pred.max(1)
if(predicted==0):
  print("False")
else:
  print("True")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


False


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [27]:
embedding = elmo.embed_sentence(sentence1.split())
embedding = np.average(embedding, axis = 0)
embedding1 = embedding[sentence1_index]

embedding = elmo.embed_sentence(sentence2.split())
embedding = np.average(embedding, axis = 0)
embedding2 = embedding[sentence1_index]
embedding1, embedding2 = torch.tensor(embedding1).type(torch.FloatTensor), torch.tensor(embedding2).type(torch.FloatTensor)
embedding1 = embedding1.unsqueeze(0)
embedding2 = embedding2.unsqueeze(0)

dnn = WiCNet(batch_size = 1).to(device)
dnn.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/checkpoints/networkwicelmo_train_epoch_57.ckpt"))

y_pred = dnn(embedding1.to(device), embedding2.to(device))
_, predicted = y_pred.max(1)
if(predicted==0):
  print("False")
else:
  print("True")


False


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [0]:
premise = "Two blond women are hugging one another." #input("Enter the premise: ")
hypothesis = "There are women showing affection." #input("Enter the hypothesis: ")

In [38]:
print("_______BERT_________")
prediction = predict_bert(premise, hypothesis)
_, prediction = prediction.max(1)
if(prediction==2):
  print("Entailment")
elif(prediction==1):
  print("Contradiction")
else:
  print("Neutral")

print("_______RoBERTa________")
prediction = predict_roberta(premise, hypothesis)
if(prediction==2):
  print("Entailment")
elif(prediction==1):
  print("Neutral")
else:
  print("Contradiction")

_______BERT_________


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


Entailment
_______RoBERTa________
Entailment


In [41]:
text = input("Enter the text to be summarized: ")

Enter the text to be summarized: Extractive text summarization involves the selection of phrases and sentences from the source document to make up the new summary. Techniques involve ranking the relevance of phrases in order to choose only those most relevant to the meaning of the source.  Abstractive text summarization involves generating entirely new phrases and sentences to capture the meaning of the source document. This is a more challenging approach, but is also the approach ultimately used by humans. Classical methods operate by selecting and compressing content from the source document.


In [0]:
x = preprocess(text)
text = tokenizer1.tokenize(x)

In [43]:
graph = {}
sentences = text
#print(sentences)
for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
      with torch.no_grad():
         prediction = predict_bert(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
result = []
while len(graph)>0:
  s1 = next(iter(graph))
  if(len(s1)<=5):
    del graph[s1]
    continue
  result.append(s1)
  f = 0
  i = 0
  s2 = graph[s1][0]
  while(len(s2)<=5):
    try:
      del graph[s2]
    except:
      pass
    i = i+1
    try:
      s2 = graph[s1][i]
    except:
      f = 1
      break
  if f == 0:
    result.append(s2)
  del graph[s1]
  try:
    del graph[s2]
  except:
    pass
result = list(dict.fromkeys(result))
result = [x[:len(x)-1] + "." for x in result]
result = " ".join(result)
print(result)

['Extractive text summarization involves the selection of phrases and sentences from the source document to make up the new summary.', 'Techniques involve ranking the relevance of phrases in order to choose only those most relevant to the meaning of the source.', 'Abstractive text summarization involves generating entirely new phrases and sentences to capture the meaning of the source document.', 'This is a more challenging approach, but is also the approach ultimately used by humans.', 'Classical methods operate by selecting and compressing content from the source document.']


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Extractive text summarization involves the selection of phrases and sentences from the source document to make up the new summary. Techniques involve ranking the relevance of phrases in order to choose only those most relevant to the meaning of the source. Abstractive text summarization involves generating entirely new phrases and sentences to capture the meaning of the source document. This is a more challenging approach, but is also the approach ultimately used by humans. Classical methods operate by selecting and compressing content from the source document.


In [46]:
graph = {}
sentences = text
#print(sentences)
for sentence1 in sentences:
   adj_list = []
   for sentence2 in sentences:
     if sentence1 == sentence2:
       continue
     s = []
     s.append(sentence1)
     s.append(sentence2)
     if len(s) != 2:
       continue
     try:
      with torch.no_grad():
         prediction = predict_bert(s[0], s[1])
         _, prediction = prediction.max(1)
         #print(prediction)
         if prediction == 2:
          #print("yes")
          adj_list.append(sentence2)
     except Exception as e:
       print(e)
     if len(adj_list) > 0:
       graph[sentence1] = adj_list
graph_list = sort_by_values_len(graph)
result = []
if len(graph)>0 :
  h = 0
  result.append(list(graph_list[h].keys())[0])
  while(dominating_set(result, sentences, graph) == 0):
    h = h + 1
    result.append(list(graph_list[h].keys())[0])
result = [x[:len(x)-1] + "." for x in result]
result = "".join(result)
print(result)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Abstractive text summarization involves generating entirely new phrases and sentences to capture the meaning of the source document.
